### Описание

Отображение графиков суточных показателей по выбранной скважине по данным из ШТР с использованием библиотеки Dash

In [ ]:
# импорт библиотек для работы с базой оданных Оракл
import sys
import cx_Oracle

In [ ]:
# импорт стандартных библиотек
from datetime import date
import socket
import threading
import sqlalchemy
import numpy as np
import pandas as pd

In [ ]:
# импорт графических библиотек
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, dash_table, State, Input, Output, exceptions
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template

### Настройка параметров подключения к БД

In [ ]:
DBDRIVER = "oracle"           # тип драйвера для подключения к БД
DBHOST = "osidb.oilcmp.ru"    # сетевое имя или IP-адрес сервера СУБД
DBPORT = 1521                 # сетевой порт сервера СУБД
DBSERVICE = "usoi"            # название службы на сервере СУБД

In [ ]:
# названия схем и ДО
DB1, DO1    = 'WELLOP1',   'Нефтегаз1'
DB2, DO2    = 'WELLOP2',   'Нефтегаз2'
DB3, DO4    = 'WELLOP3',   'Нефтегаз3'

### Настройка параметров Dash

In [ ]:
# для локального использования по ссылке http://localhost:<port>
#DASHHOST = "localhost"
#DASHPORT = "9060"
# для внешнего доступа по ссылке http://<host>:<port>
DASHHOST = socket.gethostname()
DASHPORT = "8050"
# ссылка на папку с файлом world_110m.json
plotly_cfg = { 'locale' : 'ru', 'topojsonURL' : '/assets/' }
# название темы для Plotly
#plotly_theme = None
plotly_theme = "flatly"
# настройка внешних стилей
#external_stylesheets = None
external_stylesheets = ["/assets/flatly/bootstrap.min.css", "/assets/dbc.min.css"]
# настройка внешних скриптов
#external_scripts = None
external_scripts = ['/assets/plotly-locale-ru.js']

### Инициализация текстовых констант

In [ ]:
# интервал синхронизации списка скважин и месторождений из БД (в секундах)
UPDATE_WELL_INTERVAL = 600

# названия вкладок
TAB_SINGLE_WELL = "Single"
TAB_MULTI_WELL = "Multi"

# количество параметров на графике Multi
N_MULTI_ROWS = 2
N_MULTI_COLS = 5
N_MULTI_WELL_CURVES = N_MULTI_ROWS * N_MULTI_COLS

# названия осей
AXIS_LEFT = "Левая ось"
AXIS_RIGHT = "Правая ось"

# названия столбцов
COL_DATE = "Дата"
COL_REMARKS = "Примечания"

# справочник фильтра по месторождениям
FieldFilterNone = "Все"

# справочник фильтра по скважинам
WellFilterNone = "Нет"
WellFilterProjPurpose = "По проектному назначению"
WellFilterCurrPurpose = "По текущему характеру работы"

# справочников названий назначения скважин
WellPurposeOilName     = "Нефтяные"
WellPurposeGasName     = "Газовые"
WellPurposeGascondName = "Газоконденсатные"
WellPurposeInjName     = "Нагнетательные"

# справочник текущего характера работы (назначения) скважин
WellCurrPurposeOilId     = "XR0011"
WellCurrPurposeGasId     = "XR0012"
WellCurrPurposeGascondId = "XR0015"
WellCurrPurposeInjId     = "XR0020"

# справочник проектного назначения скважин
WellProjPurposeOilId     = "NP0011"
WellProjPurposeGasId     = "NP0012"
WellProjPurposeGascondId = "NP0014"
WellProjPurposeInjId     = "NP0020"

### Запросы SQL

In [ ]:
# запрос для извлечения списка месторождений
SQL_CHESS_FIELD_LIST = ("SELECT DISTINCT A.FIELD_ID FIELD_ID, A.FIELD_NAME FIELD_NAME"
                        " FROM {0}.V_WELL_FULL A"
                        " ORDER BY A.FIELD_NAME")
# название столбца с кодом месторождения 
SQL_FIELD_ID = "FIELD_ID"
# название столбца с названием месторождения 
SQL_FIELD_NAME = "FIELD_NAME"
# запрос для извлечения списка скважин
SQL_CHESS_WELL_LIST_FULL = ("SELECT DISTINCT A.FIELD_ID FIELD_ID, A.FIELD_NAME FIELD_NAME,"
                            " A.WELL_ID WELL_ID, A.WELL_NAME WELL_NAME,"
                            " A.PROJECT_PURPOSE_ID PROJECT_PURPOSE_ID, B.PURPOSE_ID CURRENT_PURPOSE_ID"
                            " FROM {0}.V_WELL_FULL A LEFT JOIN {0}.V_WELL_FUND B ON A.WELL_ID = B.WELL_ID"
                            " WHERE B.END_DATE IS NULL"
                            " ORDER BY A.WELL_ID")
# название столбца с кодом скважины в таблице V_WELL_FULL
SQL_WELL_ID = "WELL_ID"
# название столбца с названием скважины в таблице V_WELL_FULL
SQL_WELL_NAME = "WELL_NAME"
# название столбца с кодом проектного назначения в таблице V_WELL_FUND
SQL_PROJ_PURPOSE_ID = "PROJECT_PURPOSE_ID"
# название столбца с кодом текущего характера работы в таблице V_WELL_FUND
SQL_CURR_PURPOSE_ID = "CURRENT_PURPOSE_ID"
# запрос для извлечения списка параметров с фильтром по назначению
SQL_CHESS_PARAM_LIST_PURPOSE_F = ("SELECT DISTINCT A.ID PARAM_ID, A.LONG_NAME LONG_NAME, A.SHORT_NAME SHORT_NAME "
                                  "FROM {0}.V_MEASURE_TYPE A, {0}.WELL_MEASURE B "
                                  "WHERE A.PURPOSE_ID = '{1}' "
                                  "AND A.ID = B.MEASURE_TYPE_ID "
                                  "AND A.VISIBLE = 1 "
                                  "ORDER BY A.LONG_NAME")
# название столбца с кодом параметра в таблице V_MEASURE_TYPE
SQL_PARAM_ID = "PARAM_ID"
# название столбца с полным названием параметра в таблице V_MEASURE_TYPE
SQL_PARAM_LONG_NAME = "LONG_NAME"
# название столбца с коротким названием параметра в таблице V_MEASURE_TYPE
SQL_PARAM_SHORT_NAME = "SHORT_NAME"
# запрос для извлечения суточных показателей по заданной скважине за указанный период времени
SQL_CHESS_WELL_DATA = ("SELECT MEASURE_DATE, VALUE "
                       "FROM {0}.V_WELL_MEASURE "
                       "WHERE WELL_ID = {1} AND MEASURE_TYPE_ID = {2} "
                       "AND MEASURE_DATE >= TO_DATE('{3}','dd.mm.yyyy') "
                       "AND MEASURE_DATE <= TO_DATE('{4}','dd.mm.yyyy') "
                       "ORDER BY MEASURE_DATE")
# название столбца с датой в таблице
SQL_MEASURE_DATE = "MEASURE_DATE"
# название столбца со значением параметра в таблице
SQL_MEASURE_VALUE = "VALUE"
# запрос для извлечения текстовых примечаний по заданной скважине за указанный период времени
SQL_CHESS_WELL_REMARKS = ("SELECT DT REMARKS_DATE, VALUE_TEXT "
                          "FROM {0}.V_WELL_DAILY_REMARKS "
                          "WHERE WELL_ID = {1} AND VALUE_TEXT IS NOT NULL "
                          "AND DT >= TO_DATE('{2}','dd.mm.yyyy') "
                          "AND DT <= TO_DATE('{3}','dd.mm.yyyy') "
                          "ORDER BY DT")
# название столбца с датой в таблице
SQL_REMARKS_DATE = "REMARKS_DATE"
# название столбца со значением параметра в таблице
SQL_REMARKS_TEXT = "VALUE_TEXT"

### Вспомогательные функции

In [ ]:
# функция для создания объекта подключения к базе данных
def get_db_engine(driver, user, password, host, port, service):
    sqlurl = sqlalchemy.engine.url.URL.create(driver, user, password, host, port, service)
    return sqlalchemy.engine.create_engine(sqlurl)

In [ ]:
# функция для получения списка месторождений
def get_field_df(dbengine, dbschema_id):
    sql = SQL_CHESS_FIELD_LIST.format(dbschema_id)
    field_df = pd.read_sql_query(sql=sql, con=dbengine)
    field_df.rename(columns=str.upper, inplace=True)
    field_df[SQL_FIELD_NAME] = field_df[SQL_FIELD_NAME].str.strip()
    return field_df

In [ ]:
# функция для получения списка скважин
def get_well_df(dbengine, dbschema_id):
    sql = SQL_CHESS_WELL_LIST_FULL.format(dbschema_id)
    well_df = pd.read_sql_query(sql=sql, con=dbengine)
    well_df.rename(columns=str.upper, inplace=True)
    well_df[SQL_WELL_NAME] = well_df[SQL_WELL_NAME].str.strip()
    return well_df

In [ ]:
# функция для получения списка параметров
def get_param_df(dbengine, dbschema_id, purpose_id):
    sql = SQL_CHESS_PARAM_LIST_PURPOSE_F.format(dbschema_id, purpose_id)
    param_df = pd.read_sql_query(sql=sql, con=dbengine)
    param_df.rename(columns=str.upper, inplace=True)
    param_df[SQL_PARAM_LONG_NAME] = param_df[SQL_PARAM_LONG_NAME].str.strip()
    param_df[SQL_PARAM_SHORT_NAME] = param_df[SQL_PARAM_SHORT_NAME].str.strip()
    return param_df

In [ ]:
# функция для получения массива значений параметра
def get_param_data(dbengine, dbschema_id, well_id, param_id, start_date, end_date):
    sql = SQL_CHESS_WELL_DATA.format(dbschema_id, well_id, param_id,
                                     start_date.strftime("%d.%m.%Y"), 
                                     end_date.strftime("%d.%m.%Y"))
    df = pd.read_sql_query(sql=sql, con=dbengine)
    df.rename(columns=str.upper, inplace=True)
    #df[SQL_MEASURE_DATE] = pd.to_datetime(df[SQL_MEASURE_DATE].astype(str))
    return df

In [ ]:
# функция для получения массива примечаний
def get_well_remarks(dbengine, dbschema_id, well_id, start_date, end_date):
    sql = SQL_CHESS_WELL_REMARKS.format(dbschema_id, well_id, 
                                     start_date.strftime("%d.%m.%Y"), 
                                     end_date.strftime("%d.%m.%Y"))
    df = pd.read_sql_query(sql=sql, con=dbengine)
    df.rename(columns=str.upper, inplace=True)
    #df[SQL_REMARKS_DATE] = pd.to_datetime(df[SQL_REMARKS_DATE].astype(str))
    return df

In [ ]:
# periodic task wrapper 
def periodic_task(interval, times = -1): 
    def outer_wrap(function): 
        def wrap(*args, **kwargs): 
            stop = threading.Event() 
            
            def inner_wrap(): 
                i = 0 
                while i != times and not stop.is_set(): 
                    stop.wait(interval) 
                    function(*args, **kwargs) 
                    i += 1 
                    
            t = threading.Timer(0, inner_wrap) 
            t.daemon = True 
            t.start() 
            return stop 
        return wrap 
    return outer_wrap

### Инициализация глобального хранилища данных

In [ ]:
# список схем на сервере
dbschema_dict = {
    DB1 : { 'id' : DB1, 'name' : DO1, 'user': '***', 'password': '***' },
    DB2 : { 'id' : DB2, 'name' : DO2, 'user': '***', 'password': '***' },
    DB3 : { 'id' : DB3, 'name' : DO3, 'user': '***', 'password': '***' },
}

In [ ]:
# список фильтров по скважинам
well_filter_list = [
    WellFilterNone,
    WellFilterProjPurpose,
    WellFilterCurrPurpose,  
]

# словарь проектного назначений скважин
well_proj_purpose_dict = {
    WellProjPurposeOilId     : { 'id' : WellProjPurposeOilId,     'name' : WellPurposeOilName },
    WellProjPurposeGasId     : { 'id' : WellProjPurposeGasId,     'name' : WellPurposeGasName },
    WellProjPurposeGascondId : { 'id' : WellProjPurposeGascondId, 'name' : WellPurposeGascondName },
    WellProjPurposeInjId     : { 'id' : WellProjPurposeInjId,     'name' : WellPurposeInjName },
}

# словарь текущего характера работы (назначения) скважин
well_curr_purpose_dict = {
    WellCurrPurposeOilId     : { 'id' : WellCurrPurposeOilId,     'name' : WellPurposeOilName },
    WellCurrPurposeGasId     : { 'id' : WellCurrPurposeGasId,     'name' : WellPurposeGasName },
    WellCurrPurposeGascondId : { 'id' : WellCurrPurposeGascondId, 'name' : WellPurposeGascondName },
    WellCurrPurposeInjId     : { 'id' : WellCurrPurposeInjId,     'name' : WellPurposeInjName },
}

# словарь соответствия текущего и проектного назначения
well_curr2proj_purpose_dict = {
    WellCurrPurposeOilId     : WellProjPurposeOilId,
    WellCurrPurposeGasId     : WellProjPurposeGasId,
    WellCurrPurposeGascondId : WellProjPurposeGascondId,
    WellCurrPurposeInjId     : WellProjPurposeInjId,   
}

In [ ]:
# словарь месторождений
def make_field_dict(dbschema_dict):
    field_dict = dict()

    for dbschema_id in dbschema_dict.keys():

        user = dbschema_dict[dbschema_id]['user']
        pwd = dbschema_dict[dbschema_id]['password']
        dbengine = get_db_engine(DBDRIVER, user, pwd, DBHOST, DBPORT, DBSERVICE)

        field_df = get_field_df(dbengine, dbschema_id)

        db_field_dict = { row[SQL_FIELD_ID] : { 'id'    : row[SQL_FIELD_ID],
                                                'name'  : row[SQL_FIELD_NAME],
                                              }
                          for idx, row in field_df.iterrows()}

        field_dict[dbschema_id] = db_field_dict
        
    return field_dict

In [ ]:
# словарь скважин
def make_well_dict(dbschema_dict):
    well_dict = dict()

    for dbschema_id in dbschema_dict.keys():

        user = dbschema_dict[dbschema_id]['user']
        pwd = dbschema_dict[dbschema_id]['password']
        dbengine = get_db_engine(DBDRIVER, user, pwd, DBHOST, DBPORT, DBSERVICE)

        well_df = get_well_df(dbengine, dbschema_id)

        db_well_dict = { row[SQL_WELL_ID] : { 'id'              : row[SQL_WELL_ID],
                                              'name'            : row[SQL_WELL_NAME],
                                              'field_id'        : row[SQL_FIELD_ID],
                                              'proj_purpose_id' : row[SQL_PROJ_PURPOSE_ID],
                                              'curr_purpose_id' : row[SQL_CURR_PURPOSE_ID],
                                            }
                         for idx, row in well_df.iterrows()}   

        well_dict[dbschema_id] = db_well_dict
        
    return well_dict

In [ ]:
# словарь параметров
def make_param_dict(dbschema_dict):
    param_dict = dict()

    for dbschema_id in dbschema_dict.keys():

        user = dbschema_dict[dbschema_id]['user']
        pwd = dbschema_dict[dbschema_id]['password']
        dbengine = get_db_engine(DBDRIVER, user, pwd, DBHOST, DBPORT, DBSERVICE)

        new_param_dict = dict()

        for purpose_id in well_curr_purpose_dict.keys():

            param_df = get_param_df(dbengine, dbschema_id, purpose_id)

            db_param_dict = { row[SQL_PARAM_ID] : { 'id'         : row[SQL_PARAM_ID],
                                                    'name'       : row[SQL_PARAM_LONG_NAME],
                                                    'label'      : row[SQL_PARAM_SHORT_NAME],
                                                    'purpose_id' : purpose_id,
                                                  } 
                              for idx, row in param_df.iterrows()}

            new_param_dict.update(db_param_dict)        

        param_dict[dbschema_id] = new_param_dict
        
    return param_dict

In [ ]:
# словарь отображаемых по умолчанию параметров по каждой схеме
def make_def_param_dict(dbschema_dict):
    def_param_dict = dict()

    for dbschema_id in dbschema_dict.keys():

        new_param_dict = dict()

        new_param_dict[WellCurrPurposeOilId] = [1, 4, 3]
        new_param_dict[WellCurrPurposeGasId] = [105, 107, 6110]
        new_param_dict[WellCurrPurposeGascondId] = [132, 131, 123]
        new_param_dict[WellCurrPurposeInjId] = [7086, 26, 15]

        def_param_dict[dbschema_id] = new_param_dict
        
    return def_param_dict

In [ ]:
# глобальное хранилище данных
global_data = dict()
global_data["dbschema_dict"] = dbschema_dict
global_data["well_filter_list"] = well_filter_list
global_data["well_proj_purpose_dict"] = well_proj_purpose_dict
global_data["well_curr_purpose_dict"] = well_curr_purpose_dict
global_data["well_curr2proj_purpose_dict"] = well_curr2proj_purpose_dict
global_data["field_dict"] = make_field_dict(dbschema_dict)
global_data["well_dict"] = make_well_dict(dbschema_dict)
global_data["param_dict"] = make_param_dict(dbschema_dict)
global_data["def_param_dict"] = make_def_param_dict(dbschema_dict)

In [ ]:
# колбак для синхронизации словаря скважин и месторождений из БД
@periodic_task(UPDATE_WELL_INTERVAL)
def update_well_dict():
    
    try:
        dbschema_dict = global_data["dbschema_dict"]
        global_data["field_dict"] = make_field_dict(dbschema_dict)
        global_data["well_dict"] = make_well_dict(dbschema_dict)
        
    except:
        pass

_ = update_well_dict()

### Инициализация объекта Dash

In [ ]:
# функция для отрисовки закладки Single
def make_single_well_tab():
    
    dbschema_dict = global_data["dbschema_dict"]
    well_curr_purpose_dict = global_data["well_curr_purpose_dict"]
    well_filter_list = global_data["well_filter_list"]
    
    # заполнение выпадающего списка "название предприятия/имя схемы"
    dbschema_opt = [{ 'label': dbschema_dict[key]['name'],
                      'value': dbschema_dict[key]['id']
                    } 
                    for key in dbschema_dict.keys()]
    
    # заполнение переключателя "назначение/код в БД"
    purpose_opt = [{ 'label': well_curr_purpose_dict[key]['name'],
                     'value': well_curr_purpose_dict[key]['id']
                   } 
                   for key in well_curr_purpose_dict.keys()]
    
    return [
        html.Br(),
        dbc.Row([
            dbc.Col(width=4, children=[
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.B("Выберите ДО:"),
                            dcc.Dropdown( # выпадающий список предприятий (схем в базе данных)
                                id='single-well-db-name',
                                className="dbc",
                                placeholder="",
                                options=dbschema_opt, 
                                value=dbschema_opt[0]['value'] # значение по умолчанию (имя первой схемы)
                            )
                        ]),
                        html.Div([
                            html.B("Фильтр списка скважин:"),
                            dcc.RadioItems( # переключатель характера работы
                                id='single-well-filter-name',
                                className="dbc",
                                labelStyle={'display': 'block'},
                                options=well_filter_list,
                                value=well_filter_list[0] # значение по умолчанию (нет)
                            )
                        ]),
                        html.Div([
                            html.B("Характер работы / Проектное назначение:"),
                            dcc.RadioItems( # переключатель характера работы
                                id='single-well-purpose-name',
                                className="dbc",
                                labelStyle={'display': 'block'},
                                options=purpose_opt,
                                value=purpose_opt[0]['value'] # значение по умолчанию (нефтяные)
                            )
                        ]),
                        html.Div([
                            html.B("Выберите месторождение:"),
                            dcc.Dropdown( # выпадающий список названий месторождений
                                id='single-well-field-name',
                                className="dbc",
                                placeholder=""
                            )
                        ]),
                        html.Div([
                            html.B("Выберите скважину:"),
                            dcc.Dropdown( # выпадающий список названий скважин
                                id='single-well-well-name',
                                className="dbc",
                                placeholder=""
                            )
                        ]),
                        html.Div([
                            html.B("Параметры по левой оси:"),
                            dcc.Dropdown( # выпадающий список названий параметров
                                id='single-well-left-axis-param-name',
                                className="dbc",
                                placeholder="",
                                multi=True
                            )                
                        ]),
                        html.Div([
                            html.B("Параметры по правой оси:"),
                            dcc.Dropdown( # выпадающий список названий параметров
                                id='single-well-right-axis-param-name',
                                className="dbc",
                                placeholder="",
                                multi=True
                            )                
                        ]),
                        html.Div([
                            html.B("Диапазон дат:"),
                            html.Div(style={'display': 'flex'}, children=[
                                html.Div([
                                    dcc.DatePickerSingle( # выбор начальной даты
                                        id='single-well-start-date-picker',
                                        className="dbc",
                                        date=date(date.today().year - 1, date.today().month, date.today().day),
                                        display_format='DD.MM.YYYY',
                                        first_day_of_week=1
                                    )
                                ]),
                                html.Div([
                                    html.Span(html.B('→'), style={'font-size': 30})
                                ]),
                                html.Div([
                                    dcc.DatePickerSingle( # выбор конечной даты
                                        id='single-well-end-date-picker',
                                        className="dbc",
                                        date=date.today(),
                                        display_format='DD.MM.YYYY',
                                        first_day_of_week=1
                                    )
                                ])
                            ])
                        ]),
                        html.Div([
                            dbc.Button( # кнопка
                                "Применить",
                                id='single-well-submit',
                                style={'width': '100%', 'margin-top': 10},
                                n_clicks=0
                            )
                        ])
                    ])
                ])
            ]),
            dbc.Col(width=8, children=[
                dbc.Row([
                    dbc.Col([
                        dbc.Card([
                            dbc.CardBody([
                                html.Div([
                                    dcc.Graph( # график суточных параметров скважины
                                        id='single-well-plot',
                                        config=plotly_cfg
                                    )
                                ])
                            ])
                        ])
                    ])
                ]),
                html.Br(),
                dbc.Row([
                    dbc.Col([
                        dbc.Card([
                            dbc.CardBody([
                                html.Div(id='single-well-table')
                            ])
                        ])
                    ])
                ])
            ])
        ])
    ]

In [ ]:
# функция для отрисовки настроек кривых на вкладке Multi
axis_list = [AXIS_LEFT, AXIS_RIGHT]
def_axis = axis_list[0]
trace_modes = ['lines+markers', 'markers', 'lines']
def_trace_mode = trace_modes[0]
line_dashes = ['solid', 'dot', 'dash', 'longdash', 'dashdot', 'longdashdot']
def_line_dash = line_dashes[0]
marker_symbols = ['circle', 'square', 'diamond', 'cross', 'x', 'star', 'hexagram',
                  'circle-open', 'square-open', 'diamond-open', 'cross-open', 'x-open', 'star-open', 'hexagram-open']
def_marker_symbol = marker_symbols[0]
curve_colors = ['aliceblue', 'antiquewhite', 'aqua', 'aquamarine', 'azure', 'beige', 'bisque', 'black', 'blanchedalmond', 
                'blue', 'blueviolet', 'brown', 'burlywood', 'cadetblue', 'chartreuse', 'chocolate', 'coral', 'cornflowerblue', 
                'cornsilk', 'crimson', 'cyan', 'darkblue', 'darkcyan', 'darkgoldenrod', 'darkgray', 'darkgrey', 'darkgreen', 
                'darkkhaki', 'darkmagenta', 'darkolivegreen', 'darkorange', 'darkorchid', 'darkred', 'darksalmon', 
                'darkseagreen', 'darkslateblue', 'darkslategray', 'darkslategrey', 'darkturquoise', 'darkviolet', 'deeppink', 
                'deepskyblue', 'dimgray', 'dimgrey', 'dodgerblue', 'firebrick', 'floralwhite', 'forestgreen', 'fuchsia', 
                'gainsboro', 'ghostwhite', 'gold', 'goldenrod', 'gray', 'grey', 'green', 'greenyellow', 'honeydew', 'hotpink', 
                'indianred', 'indigo', 'ivory', 'khaki', 'lavender', 'lavenderblush', 'lawngreen', 'lemonchiffon', 'lightblue', 
                'lightcoral', 'lightcyan', 'lightgoldenrodyellow', 'lightgray', 'lightgrey', 'lightgreen', 'lightpink', 
                'lightsalmon', 'lightseagreen', 'lightskyblue', 'lightslategray', 'lightslategrey', 'lightsteelblue', 
                'lightyellow', 'lime', 'limegreen', 'linen', 'magenta', 'maroon', 'mediumaquamarine', 'mediumblue', 
                'mediumorchid', 'mediumpurple', 'mediumseagreen', 'mediumslateblue', 'mediumspringgreen', 'mediumturquoise', 
                'mediumvioletred', 'midnightblue', 'mintcream', 'mistyrose', 'moccasin', 'navajowhite', 'navy', 'oldlace', 
                'olive', 'olivedrab', 'orange', 'orangered', 'orchid', 'palegoldenrod', 'palegreen', 'paleturquoise', 
                'palevioletred', 'papayawhip', 'peachpuff', 'peru', 'pink', 'plum', 'powderblue', 'purple', 'red', 'rosybrown', 
                'royalblue', 'rebeccapurple', 'saddlebrown', 'salmon', 'sandybrown', 'seagreen', 'seashell', 'sienna', 'silver',
                'skyblue', 'slateblue', 'slategray', 'slategrey', 'snow', 'springgreen', 'steelblue', 'tan', 'teal', 'thistle',
                'tomato', 'turquoise', 'violet', 'wheat', 'white', 'whitesmoke', 'yellow', 'yellowgreen']
def_curve_color = 'red'
def make_cfg_col(num):
    return dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            #html.B("Месторождение:"),
                            dcc.Dropdown( # выпадающий список названий месторождений
                                id=f'multi-well-field-name-{num}',
                                className="dbc",
                                clearable=True,
                                placeholder="Месторождение"
                            )
                        ]),
                        html.Div([
                            #html.B("Скважина:"),
                            dcc.Dropdown( # выпадающий список названий скважин
                                id=f'multi-well-well-name-{num}',
                                className="dbc",
                                clearable=True,
                                placeholder="Скважина"
                            )
                        ]),
                        html.Div([
                            #html.B("Параметр:"),
                            dcc.Dropdown( # выпадающий список названий параметров
                                id=f'multi-well-param-name-{num}',
                                className="dbc",
                                clearable=True,
                                placeholder="Параметр"
                            )                
                        ]),                  
                        html.Div([
                            #html.B("Выбор оси:"),
                            dcc.Dropdown( # выпадающий список
                                id=f'multi-well-axis-name-{num}',
                                className="dbc",
                                clearable=False,
                                placeholder="Ось",
                                options=axis_list,
                                value=def_axis
                            )                
                        ]),                  
                        html.Div([
                            #html.B("Стиль кривой:"),
                            dcc.Dropdown( # выпадающий список
                                id=f'multi-well-trace-mode-{num}',
                                className="dbc",
                                clearable=False,
                                placeholder="Стиль кривой",
                                options=trace_modes,
                                value=def_trace_mode
                            )                
                        ]),
                        html.Div([
                            html.B("Линия"),
                            dcc.Dropdown( # выпадающий список
                                id=f'multi-well-line-color-{num}',
                                className="dbc",
                                clearable=False,
                                placeholder="Цвет линии",
                                options=curve_colors,
                                value=def_curve_color
                            )                
                        ]),     
                        html.Div([
                            #html.B("Стиль линии:"),
                            dcc.Dropdown( # выпадающий список
                                id=f'multi-well-line-dash-{num}',
                                className="dbc",
                                clearable=False,
                                placeholder="Стиль линии",
                                options=line_dashes,
                                value=def_line_dash
                            )                
                        ]),     
                        html.Div([
                            #html.B("Толщина линии:"),
                            dcc.Dropdown( # выпадающий список
                                id=f'multi-well-line-width-{num}',
                                className="dbc",
                                clearable=False,
                                placeholder="Толщина линии",
                                options=[x for x in range(1, 11)],
                                value=1
                            )                
                        ]),                             
                        html.Div([
                            html.B("Маркер"),
                            dcc.Dropdown( # выпадающий список
                                id=f'multi-well-marker-color-{num}',
                                className="dbc",
                                clearable=False,
                                placeholder="Цвет маркера",
                                options=curve_colors,
                                value=def_curve_color
                            )   
                        ]),
                        html.Div([
                            #html.B("Стиль маркера:"),
                            dcc.Dropdown( # выпадающий список
                                id=f'multi-well-marker-symbol-{num}',
                                className="dbc",
                                clearable=False,
                                placeholder="Символ маркера",
                                options=marker_symbols,
                                value=def_marker_symbol
                            )                
                        ]),                  
                        html.Div([
                            #html.B("Размер маркера:"),
                            dcc.Dropdown( # выпадающий список
                                id=f'multi-well-marker-size-{num}',
                                className="dbc",
                                clearable=False,
                                placeholder="Размер маркера",
                                options=[x for x in range(1, 11)],
                                value=1
                            )                
                        ]),                             
                    ])
                ])
            ])

In [ ]:
# функция для отрисовки закладки Multi
def make_multi_well_tab():
    
    dbschema_dict = global_data["dbschema_dict"]
    
    # заполнение выпадающего списка "название предприятия/имя схемы"
    dbschema_opt = [{ 'label': dbschema_dict[key]['name'],
                      'value': dbschema_dict[key]['id']
                    } 
                    for key in dbschema_dict.keys()]
    
    # добавить настройки кривых
    cfg_rows = []
    for row in range(N_MULTI_ROWS):
        cfg_rows.append(html.Br())
        cfg_rows.append(
            dbc.Row([make_cfg_col(row * N_MULTI_COLS + col) for col in range(N_MULTI_COLS)])
        )
    
    return [
        html.Br(),
        dbc.Row([
            dbc.Col(width=4, children=[
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.B("Выберите ДО:"),
                            dcc.Dropdown( # выпадающий список предприятий (схем в базе данных)
                                id='multi-well-db-name',
                                className="dbc",
                                options=dbschema_opt, 
                                value=dbschema_opt[0]['value'] # значение по умолчанию (имя первой схемы)
                            )
                        ]),
                        html.Div([
                            html.B("Диапазон дат:"),
                            html.Div(style={'display': 'flex'}, children=[
                                html.Div([
                                    dcc.DatePickerSingle( # выбор начальной даты
                                        id='multi-well-start-date-picker',
                                        className="dbc",
                                        date=date(date.today().year - 1, date.today().month, date.today().day),
                                        display_format='DD.MM.YYYY',
                                        first_day_of_week=1
                                    )
                                ]),
                                html.Div([
                                    html.Span(html.B('→'), style={'font-size': 30})
                                ]),
                                html.Div([
                                    dcc.DatePickerSingle( # выбор конечной даты
                                        id='multi-well-end-date-picker',
                                        className="dbc",
                                        date=date.today(),
                                        display_format='DD.MM.YYYY',
                                        first_day_of_week=1
                                    )
                                ])
                            ])
                        ]),
                        html.Div([
                            dbc.Button( # кнопка
                                "Применить",
                                id='multi-well-submit',
                                style={'width': '100%', 'margin-top': 10},
                                n_clicks=0
                            )
                        ]),
                    ])
                ])
            ]),
            dbc.Col(width=8, children=[
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            dcc.Graph( # график суточных параметров скважины
                                id='multi-well-plot',
                                config=plotly_cfg
                            )                            
                        ])
                    ])
                ])
            ])
        ]),
        html.Br(),
        dbc.Row([
            dbc.Col(
                cfg_rows
            )
        ])
    ]    

In [ ]:
# функция полной отрисовки html-страницы Dash
def serve_layout():   
    single_well_tab = make_single_well_tab()
    multi_well_tab = make_multi_well_tab()
    return dbc.Container([
        html.H2("Графики суточных параметров по скважинам"),
        dbc.Tabs([
            dbc.Tab(single_well_tab, label=TAB_SINGLE_WELL),
            dbc.Tab(multi_well_tab, label=TAB_MULTI_WELL),
        ])
    ])

In [ ]:
# инициализация темы для библиотеки Plotly
if plotly_theme is not None:
    load_figure_template(plotly_theme)

# инициализация веб-приложения
app = Dash(__name__, external_stylesheets=external_stylesheets, external_scripts=external_scripts)

# для обновления страницы будет вызываться функция serve_layout
app.layout = serve_layout

### Функции обратного вызова (колбаки) для отрисовки элементов страницы Single

In [ ]:
# колбак для обновления выпадающего списка месторождений при смене схемы
@app.callback(
    [Output('single-well-field-name', 'options'),  # выходной объект: список месторождений для выпадающего списка field-name
     Output('single-well-field-name', 'value')],   # выходной объект: название выбранного по умолчанию месторождения
    [Input('single-well-db-name', 'value')]        # входной объект: выбранная схема
)
def update_field_options(dbschema_id):
    
    none_return_value = [], None
    
    if not all([dbschema_id]):
        return none_return_value
    
    field_dict = global_data["field_dict"]
    
    this_field_dict = field_dict[dbschema_id]   
    
    # если список месторождений пустой
    if len(this_field_dict) == 0:
        return [{ 'label': FieldFilterNone, 
                  'value': FieldFilterNone
               }], \
               FieldFilterNone
    
    # заполнение списка "название месторождения/код в БД"
    field_opt = [{ 'label': FieldFilterNone, 
                   'value': FieldFilterNone
                }] + \
                [{ 'label' : this_field_dict[key]['name'],
                   'value' : this_field_dict[key]['id']
                 }
                 for key in this_field_dict.keys()]
    
    # возвращаем список и значение по умолчанию для выпадающего списка месторождений
    return field_opt, field_opt[0]['value']

In [ ]:
# колбак для обновления выпадающего списка скважин
@app.callback(
    [Output('single-well-well-name', 'options'),   # выходной объект: список скважин для выпадающего списка well-name
     Output('single-well-well-name', 'value')],    # выходной объект: название выбранной по умолчанию скважины 
    [Input('single-well-db-name', 'value'),        # входной объект: выбранная схема
     Input('single-well-field-name', 'value'),     # входной объект: выбранное месторождение
     Input('single-well-filter-name', 'value'),    # входной объект: выбранный фильтр
     Input('single-well-purpose-name', 'value')]   # входной объект: выбранный характер работы
)
def update_well_options(dbschema_id, field_id, filter_name, purpose_id):
    
    none_return_value = [], None
    
    if not all([dbschema_id, field_id, filter_name, purpose_id]):
        return none_return_value
    
    field_dict = global_data["field_dict"]
    well_dict = global_data["well_dict"]
    well_curr2proj_purpose_dict = global_data["well_curr2proj_purpose_dict"]
    
    this_field_dict = field_dict[dbschema_id]   
    this_well_dict = well_dict[dbschema_id]  
    
    # если список скважин пустой
    if len(this_well_dict) == 0:
        return none_return_value
    
    # заполнение списка "название скважины/код в БД"
    well_opt = [{ 'label': this_well_dict[key]['name'] + " " + this_field_dict[this_well_dict[key]['field_id']]['name'], 
                  'value': this_well_dict[key]['id']
                } 
                for key in this_well_dict.keys() 
                if (
                       (
                           (filter_name == WellFilterNone
                           ) 
                           or 
                           (filter_name == WellFilterCurrPurpose and
                            this_well_dict[key]['curr_purpose_id'] == purpose_id
                           ) 
                           or 
                           (filter_name == WellFilterProjPurpose and
                            this_well_dict[key]['proj_purpose_id'] == well_curr2proj_purpose_dict[purpose_id]
                           )
                       ) 
                       and
                       (
                           (field_id == FieldFilterNone
                           )
                           or
                           (field_id == this_well_dict[key]['field_id']
                           )
                       )
                   )
               ]
    
    # если список скважин пустой
    if len(well_opt) == 0:
        return none_return_value
    
    # возвращаем списки и значение по умолчанию для выпадающего списка скважин
    return well_opt, well_opt[0]['value']

In [ ]:
# колбак для обновления выпадающего списка параметров
@app.callback(
    [Output('single-well-left-axis-param-name', 'options'),   # выходной объект: список параметров для выпадающего списка param-name
     Output('single-well-left-axis-param-name', 'value'),     # выходной объект: список выбранных по умолчанию параметров
     Output('single-well-right-axis-param-name', 'options'),  # выходной объект: список параметров для выпадающего списка param-name
     Output('single-well-right-axis-param-name', 'value')],   # выходной объект: список выбранных по умолчанию параметров
    [Input('single-well-db-name', 'value'),        # входной объект: выбранная схема
     Input('single-well-purpose-name', 'value')]   # входной объект: выбранный характер работы
)
def update_param_options(dbschema_id, purpose_id):
    
    none_return_value = [], None, [], None
    
    if not all([dbschema_id, purpose_id]):
        return none_return_value
    
    param_dict = global_data["param_dict"]
    def_param_dict = global_data["def_param_dict"]

    this_param_dict = param_dict[dbschema_id]   
    this_def_param_dict = def_param_dict[dbschema_id] 
    
    # заполнение списка "название параметра/код в БД"
    param_opt = [{'label': this_param_dict[key]['name'], 
                  'value': this_param_dict[key]['id']
                 } 
                 for key in this_param_dict.keys()
                 if purpose_id == this_param_dict[key]['purpose_id']
                ]
    
    # если список параметров пустой
    if len(param_opt) == 0:
        return none_return_value
    
    # возвращаем списки и значение по умолчанию для выпадающих списков параметров
    return param_opt, this_def_param_dict[purpose_id], \
           param_opt, None

In [ ]:
# колбак для обновления графика при нажатии на кнопку
@app.callback(
    Output('single-well-plot', 'figure'),           # выходной объект: график
    Output('single-well-table', 'children'),        # выходной объект: таблица
    Input('single-well-submit', 'n_clicks'),        # входной объект: кнопка
    State('single-well-db-name', 'value'),          # имя выбранной пользователем схемы из выпадающего списка db-name
    State('single-well-purpose-name', 'value'),     # выбранный характер работы
    State('single-well-well-name', 'value'),        # код выбранной пользователем скважины из выпадающего списка well-name
    State('single-well-left-axis-param-name', 'value'),   # список выбранных параметров для левой оси
    State('single-well-right-axis-param-name', 'value'),  # список выбранных параметров для правой оси
    State('single-well-start-date-picker', 'date'), # начальная дата
    State('single-well-end-date-picker', 'date')    # конечная дата
)
def update_graph(n_clicks, dbschema_id, purpose_id, well_id, 
                 left_axis_param_list, right_axis_param_list, start_date, end_date):
    
    none_return_value = go.Figure(), None
    
    if not all([n_clicks, dbschema_id, purpose_id, well_id, start_date, end_date, left_axis_param_list]):
        return none_return_value
    
    dbschema_dict = global_data["dbschema_dict"]
    well_dict = global_data["well_dict"]
    param_dict = global_data["param_dict"]
    
    this_well_dict = well_dict[dbschema_id]   
    this_param_dict = param_dict[dbschema_id]   
    
    well_name = this_well_dict[well_id]['name']
    
    start_date = date.fromisoformat(start_date)
    end_date = date.fromisoformat(end_date)
    
    date_index = pd.date_range(start=start_date, end=end_date, freq="D", inclusive='both')
    tbl_df = pd.DataFrame(data=date_index, columns=[COL_DATE])
    
    # выборка значений параметров и примечаний из БД
    try:
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        # списки подписей каждой оси
        left_axis_label_lst = []
        right_axis_label_lst = []    
        
        user = dbschema_dict[dbschema_id]['user']
        pwd = dbschema_dict[dbschema_id]['password']
        dbengine = get_db_engine(DBDRIVER, user, pwd, DBHOST, DBPORT, DBSERVICE)

        # цикл по параметрам левой оси
        for param_id in (left_axis_param_list if left_axis_param_list is not None else []):
            
            param_name = this_param_dict[param_id]['name']
            param_label = this_param_dict[param_id]['label']
                
            df = get_param_data(dbengine, dbschema_id, well_id, param_id, start_date, end_date)
            
            if len(df.index) > 0:

                left_axis_label_lst.append(param_label)
                
                # добавляем ряд значений параметра
                fig.add_trace( 
                    go.Scatter(x=df[SQL_MEASURE_DATE], 
                               y=df[SQL_MEASURE_VALUE], 
                               name=param_name, 
                               mode='markers+lines', 
                               marker_size=5, 
                               marker_symbol='circle-open'
                    ),
                    secondary_y=False # выбор левой оси
                )
        
                # добавляем столбец в таблицу
                df.columns = [COL_DATE, param_name]
                df[COL_DATE] = df[COL_DATE].dt.normalize()
                df = df.groupby(COL_DATE).agg('mean').reset_index()
                tbl_df = tbl_df.merge(df, how='left', on=COL_DATE)
                
            else:
                
                tbl_df[param_name] = np.NaN
            
        # цикл по параметрам правой оси
        for param_id in (right_axis_param_list if right_axis_param_list is not None else []):
            
            param_name = this_param_dict[param_id]['name']
            param_label = this_param_dict[param_id]['label']
            
            df = get_param_data(dbengine, dbschema_id, well_id, param_id, start_date, end_date)
            
            if len(df.index) > 0:
                
                right_axis_label_lst.append(param_label)
                
                # добавляем ряд значений параметра
                fig.add_trace( 
                    go.Scatter(x=df[SQL_MEASURE_DATE], 
                               y=df[SQL_MEASURE_VALUE], 
                               name=param_name, 
                               mode='markers+lines', 
                               marker_size=5, 
                               marker_symbol='circle-open'
                    ),
                    secondary_y=True # выбор правой оси
                )
        
                # добавляем столбец в таблицу
                df.columns = [COL_DATE, param_name]
                df[COL_DATE] = df[COL_DATE].dt.normalize()
                df = df.groupby(COL_DATE).agg('mean').reset_index()
                tbl_df = tbl_df.merge(df, how='left', on=COL_DATE)
            
            else:
                
                tbl_df[param_name] = np.NaN
            
        # выборка примечаний
        df = get_well_remarks(dbengine, dbschema_id, well_id, start_date, end_date)
        
        if len(df.index) > 0:
            
            df.columns = [COL_DATE, COL_REMARKS]
            df[COL_DATE] = df[COL_DATE].dt.normalize()
            df = df.drop_duplicates(subset=COL_DATE, keep='first')
            tbl_df = tbl_df.merge(df, how='left', on=COL_DATE)
            
        else:
                
            tbl_df[COL_REMARKS] = ""
        
    except:
        return none_return_value

          
    tbl_df[COL_DATE] = tbl_df[COL_DATE].dt.strftime("%d.%m.%Y")      
    tbl_cols = [{ 'name': i, 'id': i} for i in tbl_df.columns]
    tbl_data = tbl_df.to_dict('records')
    
    #fig.update_xaxes(title_text="Дата")
    
    fig.update_yaxes(title_text=", ".join(left_axis_label_lst).rstrip(", "), secondary_y=False)
    fig.update_yaxes(title_text=", ".join(right_axis_label_lst).rstrip(", "), secondary_y=True)
    
    fig.update_xaxes(showline=True, linecolor='black')
    fig.update_yaxes(showline=True, linecolor='black', secondary_y=False)
    fig.update_yaxes(showline=True, linecolor='black', secondary_y=True)
    
    fig.update_xaxes(ticks='outside', showgrid=True)
    fig.update_yaxes(ticks='outside', showgrid=True, secondary_y=False)
    fig.update_yaxes(ticks='outside', showgrid=False, secondary_y=True)
    
    fig.update_layout(legend=dict(orientation='h'))
    fig.update_layout(title=dict(text=well_name, x=0.5, xanchor='center'))
    
    tbl = dash_table.DataTable(data=tbl_data,
                               columns=tbl_cols,
                               fixed_columns={'headers': True, 'data': 1},
                               page_action='native',
                               page_size=31,
                               export_format='xlsx',
                               export_headers='names',
                               export_columns='visible',
                               include_headers_on_copy_paste=True,
                               style_table={"font-size": "small", "minWidth": "100%", "overflowX": "auto"}
    )
        
    # возвращаем объект figure библиотеки plotly
    return fig, tbl

### Функции обратного вызова (колбаки) для отрисовки элементов страницы Multi

In [ ]:
# колбак для обновления выпадающего списка месторождений при смене схемы
@app.callback(
    [Output(f'multi-well-field-name-{num}', 'options') for num in range(N_MULTI_WELL_CURVES)], 
    [Output(f'multi-well-field-name-{num}', 'value') for num in range(N_MULTI_WELL_CURVES)],   
    [Input('multi-well-db-name', 'value')]
)        
def update_field_options(dbschema_id):
    
    none_return_value = [] * (N_MULTI_WELL_CURVES * 2)
    
    if not all([dbschema_id]):
        return none_return_value
    
    field_dict = global_data["field_dict"]
    
    this_field_dict = field_dict[dbschema_id]  
    
    # если список месторождений пустой
    if len(this_field_dict) == 0:
        return *([{'label': FieldFilterNone, 'value': FieldFilterNone}] * N_MULTI_WELL_CURVES), \
               *([FieldFilterNone] * N_MULTI_WELL_CURVES)
    
    # заполнение списка "название месторождения/код в БД"
    field_opt = [{ 'label': FieldFilterNone, 
                   'value': FieldFilterNone
                }] + \
                [{ 'label' : this_field_dict[key]['name'],
                   'value' : this_field_dict[key]['id']
                 }
                 for key in this_field_dict.keys()]
    
    # возвращаем список и значение по умолчанию для выпадающего списка месторождений
    return *([field_opt] * N_MULTI_WELL_CURVES), \
           *([field_opt[0]['value']] * N_MULTI_WELL_CURVES)

In [ ]:
# колбаки для обновления выпадающего списка скважин
for num in range(N_MULTI_WELL_CURVES):
    @app.callback(
        Output(f'multi-well-well-name-{num}', 'options'),
        Output(f'multi-well-well-name-{num}', 'value'),
        Input('multi-well-db-name', 'value'),
        Input(f'multi-well-field-name-{num}', 'value')
    )
    def update_well_options(dbschema_id, field_id):

        none_return_value = [], None

        if not all([dbschema_id, field_id]):
            return none_return_value
        
        field_dict = global_data["field_dict"]
        well_dict = global_data["well_dict"]
    
        this_field_dict = field_dict[dbschema_id]   
        this_well_dict = well_dict[dbschema_id] 

        # если список скважин пустой
        if len(this_well_dict) == 0:
            return none_return_value
        
        # заполнение списка "название скважины/код в БД"
        well_opt = [{ 'label': this_well_dict[key]['name'] + " " + this_field_dict[this_well_dict[key]['field_id']]['name'], 
                      'value': this_well_dict[key]['id']
                    } 
                    for key in this_well_dict.keys() 
                    if (
                           (
                               (field_id == FieldFilterNone
                               )
                               or
                               (field_id == this_well_dict[key]['field_id']
                               )
                           )
                       )
                   ]
    
        # если список скважин пустой
        if len(well_opt) == 0:
            return none_return_value

        # возвращаем списки и значение по умолчанию для выпадающего списка скважин
        return well_opt, None #well_opt[0]['value']

In [ ]:
# колбаки для обновления выпадающего списка параметров
for num in range(N_MULTI_WELL_CURVES):
    @app.callback(
        Output(f'multi-well-param-name-{num}', 'options'),
        Output(f'multi-well-param-name-{num}', 'value'),
        Input('multi-well-db-name', 'value')
    )
    def update_param_options(dbschema_id):

        none_return_value = [], None

        if not all([dbschema_id]):
            return none_return_value
        
        param_dict = global_data["param_dict"]
        
        this_param_dict = param_dict[dbschema_id]   

        # заполнение списка "название параметра/код в БД"
        param_opt = [{'label': this_param_dict[key]['name'], 
                      'value': this_param_dict[key]['id']
                     } 
                     for key in this_param_dict.keys()
                    ]

        # если список параметров пустой
        if len(param_opt) == 0:
            return none_return_value

        # возвращаем списки и значение по умолчанию для выпадающего списка параметров
        return param_opt, None #param_opt[0]['value']

In [ ]:
# колбак для обновления графика при нажатии на кнопку
@app.callback(
    Output('multi-well-plot', 'figure'),            # выходной объект: график
    Input('multi-well-submit', 'n_clicks'),         # входной объект: кнопка
    State('multi-well-db-name', 'value'),           # имя выбранной пользователем схемы из выпадающего списка db-name
    State('multi-well-start-date-picker', 'date'),  # начальная дата
    State('multi-well-end-date-picker', 'date'),    # конечная дата
    [State(f'multi-well-well-name-{num}', 'value') for num in range(N_MULTI_WELL_CURVES)], 
    [State(f'multi-well-param-name-{num}', 'value') for num in range(N_MULTI_WELL_CURVES)], 
    [State(f'multi-well-axis-name-{num}', 'value') for num in range(N_MULTI_WELL_CURVES)], 
    [State(f'multi-well-trace-mode-{num}', 'value') for num in range(N_MULTI_WELL_CURVES)], 
    [State(f'multi-well-line-color-{num}', 'value') for num in range(N_MULTI_WELL_CURVES)], 
    [State(f'multi-well-line-dash-{num}', 'value') for num in range(N_MULTI_WELL_CURVES)], 
    [State(f'multi-well-line-width-{num}', 'value') for num in range(N_MULTI_WELL_CURVES)], 
    [State(f'multi-well-marker-color-{num}', 'value') for num in range(N_MULTI_WELL_CURVES)], 
    [State(f'multi-well-marker-symbol-{num}', 'value') for num in range(N_MULTI_WELL_CURVES)], 
    [State(f'multi-well-marker-size-{num}', 'value') for num in range(N_MULTI_WELL_CURVES)]
)
def update_graph(n_clicks, dbschema_id, start_date, end_date, *args):
    
    none_return_value = go.Figure()
    
    if not all([n_clicks, dbschema_id, start_date, end_date]):
        return none_return_value
    
    dbschema_dict = global_data["dbschema_dict"]
    well_dict = global_data["well_dict"]
    param_dict = global_data["param_dict"]
    
    this_well_dict = well_dict[dbschema_id]   
    this_param_dict = param_dict[dbschema_id] 
    
    start_date = date.fromisoformat(start_date)
    end_date = date.fromisoformat(end_date)
    
    # разворачивание списка параметров args
    idx = 0
    well_id_lst          = args[idx * N_MULTI_WELL_CURVES : (idx + 1) * N_MULTI_WELL_CURVES]
    idx += 1
    param_id_lst         = args[idx * N_MULTI_WELL_CURVES : (idx + 1) * N_MULTI_WELL_CURVES]
    idx += 1
    axis_lst             = args[idx * N_MULTI_WELL_CURVES : (idx + 1) * N_MULTI_WELL_CURVES]
    idx += 1
    trace_mode_lst       = args[idx * N_MULTI_WELL_CURVES : (idx + 1) * N_MULTI_WELL_CURVES]
    idx += 1
    line_color_lst       = args[idx * N_MULTI_WELL_CURVES : (idx + 1) * N_MULTI_WELL_CURVES]
    idx += 1
    line_dash_lst        = args[idx * N_MULTI_WELL_CURVES : (idx + 1) * N_MULTI_WELL_CURVES]
    idx += 1
    line_width_lst       = args[idx * N_MULTI_WELL_CURVES : (idx + 1) * N_MULTI_WELL_CURVES]
    idx += 1
    marker_color_lst     = args[idx * N_MULTI_WELL_CURVES : (idx + 1) * N_MULTI_WELL_CURVES]
    idx += 1
    marker_symbol_lst    = args[idx * N_MULTI_WELL_CURVES : (idx + 1) * N_MULTI_WELL_CURVES]
    idx += 1
    marker_size_lst      = args[idx * N_MULTI_WELL_CURVES : (idx + 1) * N_MULTI_WELL_CURVES]
    
    # выборка значений параметров и примечаний из БД
    try:
        # флаг отсутствия выбранных параметров или данных
        no_param_flag = True
        
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        left_axis_label_lst = []
        right_axis_label_lst = []  
        
        user = dbschema_dict[dbschema_id]['user']
        pwd = dbschema_dict[dbschema_id]['password']
        dbengine = get_db_engine(DBDRIVER, user, pwd, DBHOST, DBPORT, DBSERVICE)
        
        for num in range(N_MULTI_WELL_CURVES):
            
            if not all([well_id_lst[num], param_id_lst[num]]):
                continue
            
            well_id         = well_id_lst[num]
            param_id        = param_id_lst[num]
            axis_name       = axis_lst[num]
            trace_mode      = trace_mode_lst[num]
            line_color      = line_color_lst[num]
            line_dash       = line_dash_lst[num]
            line_width      = line_width_lst[num]
            marker_color    = marker_color_lst[num]
            marker_symbol   = marker_symbol_lst[num]
            marker_size     = marker_size_lst[num]
            
            well_name = this_well_dict[well_id]['name']
            param_name = this_param_dict[param_id]['name']
            param_label = this_param_dict[param_id]['label']
                
            df = get_param_data(dbengine, dbschema_id, well_id, param_id, start_date, end_date)
            
            if len(df.index) > 0:
                
                no_param_flag = False
                
                if axis_name == AXIS_LEFT:
                    left_axis_label_lst.append(param_label)
                elif axis_name == AXIS_RIGHT:
                    right_axis_label_lst.append(param_label)
                else:
                    pass
                
                # добавляем ряд значений параметра
                fig.add_trace( 
                    go.Scatter(x=df[SQL_MEASURE_DATE], 
                               y=df[SQL_MEASURE_VALUE], 
                               name=well_name + " " + param_name, 
                               mode=trace_mode, 
                               line_color=line_color,
                               line_dash=line_dash,
                               line_width=line_width,
                               marker_color=marker_color,
                               marker_symbol=marker_symbol,
                               marker_size=marker_size
                    ),
                    secondary_y=(axis_name == AXIS_RIGHT) # выбор оси
                )
        
    except:
        return none_return_value
    
    if no_param_flag:
        return none_return_value
    
    #fig.update_xaxes(title_text="Дата")
    
    fig.update_yaxes(title_text=", ".join(left_axis_label_lst).rstrip(", "), secondary_y=False)
    fig.update_yaxes(title_text=", ".join(right_axis_label_lst).rstrip(", "), secondary_y=True)
    
    fig.update_xaxes(showline=True, linecolor='black')
    fig.update_yaxes(showline=True, linecolor='black', secondary_y=False)
    fig.update_yaxes(showline=True, linecolor='black', secondary_y=True)
    
    fig.update_xaxes(ticks='outside', showgrid=True)
    fig.update_yaxes(ticks='outside', showgrid=True, secondary_y=False)
    fig.update_yaxes(ticks='outside', showgrid=False, secondary_y=True)
    
    fig.update_layout(legend=dict(orientation='h'))
    fig.update_layout(title=dict(text="Title", x=0.5, xanchor='center'))
    
    # возвращаем объект figure библиотеки plotly
    return fig

### Запуск веб-сервера

In [ ]:
# запуск веб-сервера 
# jupyter_mode='inline'     - отображение дашборда в ячейке ноутбука (режим по умолчанию)
# jupyter_mode='external'   - без отображения, но с доступом по ссылке http://<host>:<port>
# jupyter_mode='tab'        - отображение в новой вкладке браузера
# jupyter_mode='jupyterlab' - отображение в отдельной вкладке JupyterLab (если установлен)
app.run(host=DASHHOST, port=DASHPORT, jupyter_mode='external')